# Milestone 1 - Model Exploration

In [1]:
# TODO
# 0. Load medical relation dataset
# 1. replace the entities with ENTITY1 and ENTITY2  (useful for N-gram modelling)
# 2. preprocess: tokenize -> lower case -> stop word removal -> stemming
# 3. features: bag of words (1-gram, 2-gram, 3-gram), maybe syntactic info (e.g. bag of parse tree tags between the 2 entities)
# 4. train/val/test split -> train a NB or SVM classifier
# 5. evaluate (quant: prec/recall/f1, qual: wrong predicted inputs)

In [2]:
import nltk
from nltk.corpus import reuters

nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')

ids = reuters.fileids()
reuters_ids_train = list(filter(lambda doc: doc.startswith('train'), ids))
reuters_ids_test = list(filter(lambda doc: doc.startswith('test'), ids))

reuters_raw_train = [reuters.raw(doc_id) for doc_id in reuters_ids_train]
reuters_raw_test = [reuters.raw(doc_id) for doc_id in reuters_ids_test]

reuters_categs_train = [reuters.categories(doc_id) for doc_id in reuters_ids_train]
reuters_categs_test = [reuters.categories(doc_id) for doc_id in reuters_ids_test]

print(str(len(ids)) + " documents")
print(str(len(reuters_raw_train)) + " total train documents")
print(str(len(reuters_raw_test)) + " total test documents")
print(str(len(reuters.categories())) + " categories")

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


10788 documents
7769 total train documents
3019 total test documents
90 categories
